# RPC

- 참고 (2018년 국문 번역문) : https://blog.storyg.co/rabbitmqs/tutorials/python/06-RPC

- 참고 (최신 정식 문서) : https://www.rabbitmq.com/tutorials/tutorial-six-python.html

- 참고 : https://github.com/rabbitmq/rabbitmq-tutorials


두 번째 포스트에서는 작업 큐를 사용하여 시간이 많이 소요되는 작업을 여러 작업자에게 배포하는 방법을 배웠습니다. 그러나 원격 컴퓨터에서 함수를 실행하고 결과를 기다릴 필요가 있다면 어떻게 해야 할까요? 그건 다른 이야기입니다. 이 패턴은 일반적으로 원격 프로 시저 호출 또는 RPC로 알려져 있습니다. 이 포스트에서는 RabbitMQ를 사용하여 클라이언트와 확장 가능한 RPC 서버인 RPC 시스템을 구축 할 것입니다. 예제에서 피보나치 숫자를 반환하는 더미 RPC 서비스를 만들 것입니다.


In [1]:
! rabbitmqctl --version

3.10.7


## rpc_server.py
- https://www.rabbitmq.com/tutorials/tutorial-six-python.html

In [2]:
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='rpc_queue')

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

def on_request(ch, method, props, body):
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='rpc_queue', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Sent '-f':'/Users/jpark/Library/Jupyter/runtime/kernel-175297e0-39be-496f-9295-c9012667278b.json'


## receive_logs_topic.py 
- https://www.rabbitmq.com/tutorials/tutorial-six-python.html

In [ ]:
#!/usr/bin/env python
import pika
import sys

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))
channel = connection.channel()

channel.exchange_declare(exchange='topic_logs', exchange_type='topic')

result = channel.queue_declare('', exclusive=True)
queue_name = result.method.queue

binding_keys = sys.argv[1:]
if not binding_keys:
    sys.stderr.write("Usage: %s [binding_key]...\n" % sys.argv[0])
    sys.exit(1)

for binding_key in binding_keys:
    channel.queue_bind(
        exchange='topic_logs', queue=queue_name, routing_key=binding_key)

print(' [*] Waiting for logs. To exit press CTRL+C')


def callback(ch, method, properties, body):
    print(" [x] %r:%r" % (method.routing_key, body))


channel.basic_consume(
    queue=queue_name, on_message_callback=callback, auto_ack=True)

channel.start_consuming()


## 사용예시

```bash
python receive_logs_topic.py "#"
python receive_logs_topic.py "kern.*"
python receive_logs_topic.py "*.critical"
python receive_logs_topic.py "kern.*" "*.critical"
python emit_log_topic.py "kern.critical" "A critical kernel error"
```




In [2]:
! rabbitmqctl list_exchanges

Listing exchanges for vhost / ...
name	type
amq.headers	headers
amq.match	headers
amq.rabbitmq.trace	topic
amq.direct	direct
amq.topic	topic
amq.fanout	fanout
	direct


In [5]:
! rabbitmqadmin list exchanges

+--------------------+---------+
|        name        |  type   |
+--------------------+---------+
|                    | direct  |
| amq.direct         | direct  |
| amq.fanout         | fanout  |
| amq.headers        | headers |
| amq.match          | headers |
| amq.rabbitmq.trace | topic   |
| amq.topic          | topic   |
+--------------------+---------+


In [6]:
! rabbitmqctl list_bindings

Listing bindings for vhost /...
source_name	source_kind	destination_name	destination_kind	routing_key	arguments
	exchange	task_queue	queue	task_queue	[]
	exchange	hello	queue	hello	[]


In [7]:
! rabbitmqadmin list bindings

+--------+-------------+-------------+
| source | destination | routing_key |
+--------+-------------+-------------+
|        | hello       | hello       |
|        | task_queue  | task_queue  |
+--------+-------------+-------------+
